# How to use the PiDAQ 

## Import required modules

In [ ]:
# PyDAQ modules
import pydaqhat.pydaqhat as pd
from pydaqhat.channel import Channel

# Plotting modules
from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook
from bokeh.palettes import Category20

# FLAC modules
import os.path
import numpy as np
import soundfile as sf 
from mutagen.flac import FLAC
from datetime import datetime

## Configure individual channel parameters

In [ ]:
# This is the list of channels to use for the recording
# They are of class 'Channel' and are in the format:
# Channel(channel number, name, sensitivity, IEPE enable)

# Channels can be removed from list if not required

channels=[Channel(0,"Channel 0",3000,False),
          Channel(1,"Channel 1",1234,True),
          Channel(2,"Channel 2",4321,False),
          Channel(3,"Custom name",3000,False),
          Channel(4,"Channel 4",3000,False),
          Channel(5,"Channel 5",3000,False)]

## Perform finite scan
A finite scan is a recording of pre-defined length

In [ ]:
scanresult = pd.finite_scan(
    channels = channels,  # Individual channel parameters
    sample_rate = 24000,  # Sample rate in Hz {default: 24000}
    recording_length = 1, # Length of recording in seconds {default: 1}
    verbose = False,      # Verbose text output {default: False}
)

# The data can be accessed as scanresult.data
# e.g. Channel 0 data is scanresult.data[0]

# Print channel data
for i in range(len(channels)):
    name = channels[i].name
    channel_number = channels[i].channel
    data = scanresult.data[i]
    
    print(f"""({channel_number}) {name} data: 
    {str(data)}""")


## Plot data

In [ ]:
#Set up legend titles
legend = []
for i in range(len(channels)):
    name = channels[i].name
    channel_number = channels[i].channel
    legend.append(f"({channel_number}) {name}")
    

#Plot Data
plot = figure(
    title = "Finite Recording",
    x_axis_label = "Samples",
    y_axis_label = "Unit"
)

palette=Category20[9]
for i in range(len(scanresult.data)):      
    plot.line(
        range(len(scanresult.data[i])), 
        scanresult.data[i],
        color=palette[i],
        line_width = 2,
        legend_label=legend[i]
    )

plot.legend.click_policy="hide"

show(plot, notebook_handle=True)

## Save recording as a FLAC

In [ ]:
date = datetime.now().strftime("%d_%m_%Y %H:%M:%S")
fname = f"data/{len(scanresult.data)}Ch_Recording_{date}.flac"
print(f"Saving to {fname}")
stream_data = np.transpose(np.array(scanresult.data))
sf.write(fname,stream_data, scanresult.inputs[0])

file = FLAC(fname)
file["Title"] = "Finite Recording"
#TODO create Channels class so it is possible to run MCC_data.channels.format(0)
for channel in scanresult.inputs[2]:
    file[f"Channel {channel.channel}"] = f"Enabled. Sensitivity {channel.sensitivity}. IEPE Enable {channel.iepe_enable}"
file["Number of hats"] = str(len(scanresult.channels))
file.save()